<h1>Model Training</h1>

In this notebook, we will use the Amazon SageMaker open source XGBoost container (https://github.com/aws/sagemaker-xgboost-container) to train a simple binary classification model, using the pre-processed data generated in the previous step by the processing job.
Using XGBoost as a framework provides more flexibility than using it as a built-in algorithm as it enables more advanced scenarios that allow pre-processing and post-processing scripts or any kind of custom logic to be incorporated into your training script.

Let's define the variables first.

In [1]:
# When using Amazon SageMaker Studio, please set this variable to True and execute the cell
use_sm_studio = True
if use_sm_studio:
    %cd /root/end-to-end-ml-sm/03_train_model/

/root/end-to-end-ml-sm/03_train_model


In [2]:
# Check SageMaker Python SDK version
import sagemaker
print(sagemaker.__version__)

def versiontuple(v):
    return tuple(map(int, (v.split("."))))

if versiontuple(sagemaker.__version__) < versiontuple('2.5.0'):
    raise Exception("This notebook requires at least SageMaker Python SDK version 2.5.0. Please install it via pip.")

2.23.5


In [3]:
import boto3
import sagemaker

role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sagemaker_session = sagemaker.Session()
bucket_name = sagemaker_session.default_bucket()
prefix = 'endtoendmlsm'

print(region)
print(role)
print(bucket_name)

us-east-1
arn:aws:iam::041631420165:role/service-role/AmazonSageMaker-ExecutionRole-endtoendml
sagemaker-us-east-1-041631420165


In [4]:
%store -r experiment_name
%store -r trial_name

print(experiment_name)
print(trial_name)

end-to-end-ml-sagemaker-1612336463
sklearn-xgboost-1612336478


## Training

The training code is implemented in the `source_dir/training.py` file.

The script parses arguments that are passed when the XGBoost Docker container code invokes the script for execution. These arguments represent the hyperparameters that you specify when strarting the training job plus the location of training and validation data. Then, we load training and validation data and execute XGBoost training with the provided parameters.

<strong>Note</strong>: this behavior, named Script Mode execution, is enabled by a library that is installed in the XGBoost container (sagemaker-containers, https://github.com/aws/sagemaker-containers) and facilitates the development of SageMaker-compatible Docker containers.

In [5]:
!pygmentize source_dir/training.py

import argparse
import json
import os
import random
import pandas as pd
import glob
import pickle as pkl

import xgboost

def parse_args():

    parser = argparse.ArgumentParser()

    parser.add_argument("--max_depth", type=int, default=5)
    parser.add_argument("--eta", type=float, default=0.05)
    parser.add_argument("--gamma", type=int, default=4)
    parser.add_argument("--min_child_weight", type=int, default=6)
    parser.add_argument("--silent", type=int, default=0)
    parser.add_argument("--objective", type=str, default="reg:logistic")
    parser.add_argument("--num_round", type=int, default=10)
    
    parser.add_argument('--train', type=str, default=os.environ.get('SM_CHANNEL_TRAIN'))
    parser.add_argument('--validation', type=str, default=os.environ.get('SM_CHANNEL_VALIDATION'))

    args = parser.parse_args()

    return args

def main():

    args = parse_args()
    train_files_path, validation_files_path = args.train, args.validation
    
    train_features_path = o

Once we have our script ready, we can leverage on the XGBoost estimator of the Amazon SageMaker Python SDK to start training.

In [6]:
from sagemaker.xgboost import XGBoost

hyperparameters = {
    "max_depth": "3",
    "eta": "0.1",
    "gamma": "6",
    "min_child_weight": "6",
    "silent": "0",
    "objective": "reg:logistic",
    "num_round": "20"
}

entry_point='training.py'
source_dir='source_dir/'
output_path = 's3://{0}/{1}/output/'.format(bucket_name, prefix)
code_location = 's3://{0}/{1}/code'.format(bucket_name, prefix)

estimator = XGBoost(
    base_job_name="end-to-end-ml-sm-xgb",
    entry_point=entry_point,
    source_dir=source_dir,
    output_path=output_path,
    code_location=code_location,
    hyperparameters=hyperparameters,
    instance_type="ml.m5.xlarge", # Specify local as instance type to run local-mode training
    instance_count=1,
    framework_version="0.90-2",
    py_version="py3",
    role=role
)

In [7]:
# Experiment tracking configuration
experiment_config={
    "ExperimentName": experiment_name,
    "TrialName": trial_name,
    "TrialComponentDisplayName": "xgboost-training",
}

train_config = sagemaker.TrainingInput('s3://{0}/{1}/data/preprocessed/train/'.format(
    bucket_name, prefix), content_type='text/csv')
val_config = sagemaker.TrainingInput('s3://{0}/{1}/data/preprocessed/val/'.format(
    bucket_name, prefix), content_type='text/csv')

estimator.fit({'train': train_config, 'validation': val_config },
              experiment_config=experiment_config)

2021-02-03 07:34:18 Starting - Starting the training job...
2021-02-03 07:34:40 Starting - Launching requested ML instancesProfilerReport-1612337657: InProgress
......
2021-02-03 07:35:41 Starting - Preparing the instances for training......
2021-02-03 07:36:42 Downloading - Downloading input data......
2021-02-03 07:37:46 Training - Training image download completed. Training in progress..INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Invoking user training script.

2021-02-03 07:38:17 Uploading - Uploading generated training modelINFO:sagemaker-containers:Module training does not provide a setup.py. 
Generating setup.py
INFO:sagemaker-containers:Generating setup.cfg
INFO:sagemaker-containers:Generating MANIFEST.in
INFO:sagemaker-containers:Installing module with the following command:
/miniconda3/bin/python -m pip install . 
Processing 

### Experiment analytics

You can visualize experiment analytics either from Amazon SageMaker Studio Experiments plug-in or using the SDK from a notebook, as follows:

In [ ]:
from sagemaker.analytics import ExperimentAnalytics

analytics = ExperimentAnalytics(experiment_name=experiment_name)
analytics.dataframe()

After the training is completed, the serialized model will be saved in the S3 `output_location` defined above.
You can now move to the next notebook in the **04_deploy_model** folder to see how to use that model for inference.